In [ ]:
import pandas as pd
import sys
import os

pd.set_option('display.max_columns', 100)
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)


In [ ]:
os.chdir('/.mounts/labs/reimandlab/private/users/k2zhang/TIA18674.20211202/cpdb_tutorial')
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)


cpdb_file_path = '/.mounts/labs/reimandlab/private/users/k2zhang/TIA18674.20211202/cpdb_tutorial/db/test/v5.0.0/cellphonedb.zip'
meta_file_path = '/.mounts/labs/reimandlab/private/users/k2zhang/TIA18674.20211202/Ai_10xG_library/Ai_10xG_library.metrics_summary.csv'
counts_file_path = '/.mounts/labs/reimandlab/private/users/k2zhang/TIA18674.20211202/cpdb_tutorial/counts_data.h5ad'
# microenvs_file_path = 'data/microenvironment.tsv'
out_path = '/.mounts/labs/reimandlab/private/users/k2zhang/TIA18674.20211202/cpdb_tutorial/results/method1'

metadata = pd.read_csv(meta_file_path, sep = ',')
metadata.head(3)



In [15]:
import h5py

# Open and inspect the HDF5 file
with h5py.File(counts_file_path, 'r') as f:
    def print_structure(name, obj):
        print(f"{name}: {obj}")
    f.visititems(print_structure)


matrix: <HDF5 group "/matrix" (6 members)>
matrix/barcodes: <HDF5 dataset "barcodes": shape (8331,), type "|S18">
matrix/data: <HDF5 dataset "data": shape (26202101,), type "<i4">
matrix/features: <HDF5 group "/matrix/features" (5 members)>
matrix/features/_all_tag_keys: <HDF5 dataset "_all_tag_keys": shape (1,), type "|S6">
matrix/features/feature_type: <HDF5 dataset "feature_type": shape (36602,), type "|S15">
matrix/features/genome: <HDF5 dataset "genome": shape (36602,), type "|S31">
matrix/features/id: <HDF5 dataset "id": shape (36602,), type "|S15">
matrix/features/name: <HDF5 dataset "name": shape (36602,), type "|S17">
matrix/indices: <HDF5 dataset "indices": shape (26202101,), type "<i8">
matrix/indptr: <HDF5 dataset "indptr": shape (8332,), type "<i8">
matrix/shape: <HDF5 dataset "shape": shape (2,), type "<i4">


In [16]:
# Read the HDF5 file using scanpy
adata = sc.read_10x_h5(counts_file_path)
print(adata)

# Save the AnnData object to disk if necessary
adata.write('counts_data.h5ad')

AnnData object with n_obs × n_vars = 8331 × 36602
    var: 'gene_ids', 'feature_types', 'genome'


In [17]:
from cellphonedb.src.core.methods import cpdb_analysis_method

cpdb_results = cpdb_analysis_method.call(
    cpdb_file_path = cpdb_file_path,           # mandatory: CellphoneDB database zip file.
    meta_file_path = meta_file_path,           # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,       # mandatory: normalized count matrix - a path to the counts file, or an in-memory AnnData object
    counts_data = 'hgnc_symbol',               # defines the gene annotation in counts matrix.
    # microenvs_file_path = microenvs_file_path, # optional (default: None): defines cells per microenvironment.
    score_interactions = True,                 # optional: whether to score interactions or not. 
    output_path = out_path,                    # Path to save results    microenvs_file_path = None,
    separator = '|',                           # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    threads = 5,                               # number of threads to use in the analysis.
    threshold = 0.1,                           # defines the min % of cells expressing a gene for this to be employed in the analysis.
    result_precision = 3,                      # Sets the rounding for the mean values in significan_means.
    debug = False,                             # Saves all intermediate tables emplyed during the analysis in pkl format.
    output_suffix = None                       # Replaces the timestamp in the output files by a user defined string in the  (default: None)
)


[ ][CORE][25/06/24-15:18:42][INFO] [Non Statistical Method] Threshold:0.1 Precision:3
Reading user files...


ValueError: Dataset(s) incompatible with Pandas data types, not table, or no datasets found in HDF5 file.